In [ ]:
# Import necessary libraries
from pyspark import SparkContext
import pandas as pd  # Importing pandas from the Conda environment

# Create SparkContext
sc = SparkContext()

# Load data from HDFS into RDD
rdd = sc.textFile("hdfs://localhost:9000/data/india-news-headlines_copy_1.csv")

In [ ]:
# Define your processing functions
def process_map(line):
    # Perform map operation
    return line.split(',')

def process_flatMap(line):
    # Perform flatMap operation
    return line.split(',')

def process_reduceByKey(line):
    # Perform reduceByKey operation
    return (line[1], 1)

In [ ]:
# Apply RDD transformations
rdd_map = rdd.map(process_map)
rdd_flatMap = rdd.flatMap(process_flatMap)
rdd_reduceByKey = rdd_map.map(process_reduceByKey).reduceByKey(lambda x, y: x + y)

In [ ]:
# Display results with error handling
print("RDD with map transformation:")
try:
    print(rdd_map.take(5))
except Exception as e:
    print("An error occurred while executing take(5) on RDD with map transformation:", str(e))

print("\nRDD with flatMap transformation:")
try:
    print(rdd_flatMap.take(10))
except Exception as e:
    print("An error occurred while executing take(10) on RDD with flatMap transformation:", str(e))

print("\nRDD with reduceByKey transformation:")
try:
    print(rdd_reduceByKey.take(5))
except Exception as e:
    print("An error occurred while executing take(5) on RDD with reduceByKey transformation:", str(e))

# Stop SparkContext
sc.stop()